In [1]:
from __future__ import division
import os.path as osp
import sys
import argparse
from tqdm import tqdm
import numpy as np

import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torch.nn.modules.batchnorm import BatchNorm2d 

from config import config
from dataloader import get_train_loader
from network import CrfRnnNet
from datasets import Cil

from utils.init_func import init_weight, group_weight
from engine.engine import Engine

In [2]:
criterion = nn.CrossEntropyLoss(reduction='mean',
                                    ignore_index=-1)
model = CrfRnnNet(config.num_classes, criterion=criterion,
               pretrained_model=config.pretrained_model,
               norm_layer=BatchNorm2d)
base_lr = config.lr

In [27]:
# initialize parameters
init_weight(model.psp.business_layer, nn.init.kaiming_normal_,
            BatchNorm2d, config.bn_eps, config.bn_momentum,
            mode='fan_in', nonlinearity='relu')


# group weight initialization on all layers
params_list = []
params_list = group_weight(params_list, model.psp.backbone,
                           BatchNorm2d, base_lr)
for module in model.psp.business_layer:
    params_list = group_weight(params_list, module, BatchNorm2d,
                               base_lr * 10)
params_list.append(
    dict(params=list(model.crfrnn.parameters()), weight_decay=.0, lr=base_lr))